<h1 style="font-size:2.2em; font-weight:bold; text-align:center;">Exploratory Data Analysis</h1>

This project serves as a **Proof of Concept (PoC)** to explore **anomaly scoring** techniques for detecting potentially fraudulent banking transactions.

The data covers the period from **April 2, 2025** to **April 4, 2025**, based on the assumption that in mid-2025, trade tensions lead to increased abnormal or suspicious transaction activities.

For exploratory data analysis (EDA), we will:

- Integrate dynamic currency conversion using the Riksbanken API.

- Explore missing values.

- Conduct descriptive analytics using SQL.

- Perform univariate and bivariate analysis to analyze data distribution and identify outliers.





| **Key Findings**                                                                                 |
|--------------------------------------------------------------------------------------------------|
| Dataset merged from `transaction records`, `KYC customer profiles`, and `customer summary statistics`. |
| Contains 10,716 transactions with 40 features (31 categorical, 9 numerical).                      |
| Includes 107 duplicate records.                                                                  |
| All amounts in different currencies have been converted to EUR.                                  |
| Missing values are present in some categorical features.                                         |
| Several numerical features contain potential outliers.             |
| Categorical features exhibit a mix of low and high cardinality.     |       





## <h2 style="font-size: 1.6em; font-weight: bold;"> 1. Data Collection </h2>
Import Required Packages and Data

**Importing Pandas, Numpy, Matplotlib, Seaborn, Plotly.express and Warings Library.**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
import plotly.express as px
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

**Import the CSV Data as Pandas DataFrame**

In [ ]:
#Import data from 3 reports and merge them using a left join on `customer_id`

transaction_df = pd.read_csv('data/transactions.csv')
customer_df = pd.read_csv('data/customers.csv')
sum_statistic_df = pd.read_csv('data/customer_summary_statistics.csv')

df = transaction_df.merge(customer_df, on='customer_id', how='left') \
                   .merge(sum_statistic_df, on='customer_id', how='left')

Handle duplicate columns after merging.

In [ ]:
dup_cols = []
cols = df.columns.tolist()

for i in range(len(cols)):
    for j in range(i + 1, len(cols)):
        if df[cols[i]].equals(df[cols[j]]):
            dup_cols.append((cols[i], cols[j]))
print("Columns with identical values:")
for c1, c2 in dup_cols:
    print(f"{c1} and {c2}")
    
dropped = set()
kept = set()

for c1, c2 in dup_cols:
    if c2 not in dropped:
        df.drop(columns=c2, inplace=True)
        dropped.add(c2)
        kept.add(c1)     
print("Dropped columns:", list(dropped))
print ("Remaining columns:", list(kept) )

In [ ]:
df.rename(columns={
    'available_balance_x': 'available_balance',
    'country_x' : 'country',
    'currency_x': 'currency',
    'credit_limit_x' : 'credit_limit'
}, inplace=True)

**Dataset Overview**

In [ ]:
df.shape

In [ ]:
df.dtypes.value_counts()

We have a dataset with 10,716 records and 40 columns:

  - Categorical features: 31

  - Numeric features: 9

## <h2 style="font-size: 1.6em; font-weight: bold;"> 2. Data Cleaning </h2>

- Removed Duplicate Entries

- Converted amounts to EUR using exchange rates from the Riskbanken open API

In [ ]:
df.duplicated().sum()

The data has 107 duplicate records. Drop all duplicate entries.

In [ ]:
#Remove Duplicate
df.drop_duplicates(inplace=True)
df.duplicated().sum()

Standardize all amount-related columns to EUR to ensure consistency for future analysis.

*Using dynamic exchange rates from the Riksbanken API.* 

In [ ]:
from datetime import datetime, timedelta
import urllib.request, json


# Ensure all currency codes are uppercase
currencies = df['currency'].unique()

# Build currency-to-series map
currency_to_series = {
    cur: f'SEK{cur}PMI' if cur != 'SEK' else 'SEK'
    for cur in currencies
}

rate_dict = {}

# Loop to fetch exchange rates
for cur in currencies:
    if cur == 'EUR':
        rate_dict[cur] = 1.0  # EUR to EUR
        continue
    if cur == 'SEK':
        series1 = 'SEK'
    else:
        series1 = currency_to_series[cur]

    series1 = currency_to_series[cur]
    url = f'https://api.riksbank.se/swea/v1/CrossRates/{series1}/SEKEURPMI/2025-04-4/2025-04-4' 
    try:
        req = urllib.request.Request(url)
        req.get_method = lambda: 'GET'
        response = urllib.request.urlopen(req)
        content = response.read()
        data = json.loads(content)

        # Handle different JSON structures
        if isinstance(data, list) and data:
            rate = data[0]['value']
            rate_dict[cur] = float(rate)
        elif isinstance(data, dict) and 'value' in data:
            rate_dict[cur] = float(data['value'])
        else:
            rate_dict[cur] = 1.0

    except Exception as e:
        print(f"Error fetching rate for {cur}: {e}")
        rate_dict[cur] = 1.0

cols_to_convert = [
    'amount', 'available_balance', 'value_dated_balance', 'credit_limit',
    'total_volume', 'avg_amount', 'max_amount', 'min_amount'
]
cols_after_convert = []

for col in cols_to_convert:
    new_col = col + '_eur'
    df[new_col] = df.apply(
        lambda x: float(x[col]) if x['currency'].upper() == 'EUR'
        else (
            float(x[col]) / rate_dict.get('SEK', 1.0) if x['currency'].upper() == 'SEK'
            else float(x[col]) * rate_dict.get(x['currency'].upper(), 1.0)
        ),
        axis=1
    )
    cols_after_convert.append(new_col)

In [ ]:
print (cols_after_convert)

In [ ]:
df.drop(columns=cols_to_convert, inplace=True)

In [ ]:
df.to_csv('data/RAW_transaction_monitoring_merged.csv',index=False)

## <h2 style="font-size: 1.6em; font-weight: bold;"> 3. EDA </h2>

- Explored missing values.

- Conducted descriptive analytics using SQL.

- Performed univariate and bivariate analysis.

- Analyzed data distribution and identified outliers.


**3.1 Missing Values**

In [ ]:
missing_pct = round(df.isnull().sum()*100/len(df),2)
missing_column_pct = missing_pct[missing_pct> 0.1]
missing_column_pct

We have a high percentage of missing values, mostly in categorical features. So, during feature engineering, we will replace them with a new label.

**3.2 Descriptive Analytics using SQL** 

In [ ]:
import sqlite3

con = sqlite3.connect(':memory:')
df.to_sql('RAW_transaction_monitoring_merged', con, index=False)

In [ ]:
query = '''
select count (*) as total_transactions , count(distinct customer_id) as total_customers,
count(distinct currency) as currencies, Min(amount_eur) as Min_transaction, Max(amount_eur) as Max_transaction,
AVG (amount_eur) as avg_transaction
from RAW_transaction_monitoring_merged
'''
df_result = pd.read_sql_query(query,con)
df_result

In [ ]:
query = '''
select 'Lowest_traffic_weekday' as category, weekday, transaction_count
from(
select strftime ('%w',transaction_date) as weekday, count(*) as transaction_count
from RAW_transaction_monitoring_merged 
group by strftime ('%w',transaction_date) 
order by count(*) asc
limit 1)


UNION ALL 
select 'Highest_traffic_weekday' as category, weekday, transaction_count
from(
select strftime ('%w',transaction_date) as weekday, count(*) as transaction_count
from RAW_transaction_monitoring_merged 
group by strftime ('%w',transaction_date) 
order by count(*) desc
limit 1)

'''

df_result = pd.read_sql_query(query,con)
df_result

**Descriptive Overview:**

- Total Records: **10,609** transactions

- Customers: **1,000** unique customer IDs

- Currencies: 4 types used

- Minimum Transaction Amount: €1.1

- Maximum Transaction Amount: €19,659.95

- Average Transaction Amount: €2. 618.77

- Transactions had the lowest traffic on Wednesday and the highest traffic on Thursday.


In [ ]:
query= '''
select 'Least_frequent_customer' as category, customer_id, transaction_count, total_amount
from 
(select *, sum(amount_eur) as total_amount, count(*) as transaction_count
from RAW_transaction_monitoring_merged
group by customer_id
order by count(*) asc
limit 1
)

UNION ALL

select 'Most_frequent_customer' as category, customer_id, transaction_count, total_amount
from 
(select *, sum(amount_eur) as total_amount,count(*) as transaction_count
from RAW_transaction_monitoring_merged
group by customer_id
order by count(*) desc
limit 1
)

'''
df_result = pd.read_sql_query(query,con)
df_result

In [ ]:
query = '''
select avg_amount_per_customer, rank
from
(select AVG(amount_eur) AS avg_amount_per_customer, rank() OVER(Order by  AVG(amount_eur) desc) as rank
FROM RAW_transaction_monitoring_merged
group by customer_id
order by  AVG(amount_eur) desc)
where rank = 1
or rank = 1000
'''

df_result = pd.read_sql_query(query,con)
df_result

**Customer Behavior Insights:**

   - Each customer made between 1 and 23 transactions.
   
   - Customer `CUST00273` made the least frequent transaction (1) with an amount of €210, while `CUST00797` made the most frequent transactions (23) with a total of €155,607.

   - Average transaction amount per customer ranges from approximately €18 to €14,306.

**3.3 Univariate and Bivariate analysis**

**Numeric Features (Continuous)**

In [ ]:
continuous_feature = df.select_dtypes(include='float').columns.to_list()
print(continuous_feature)

In [ ]:
#Let’s explore these Continuous features further.
plt.figure(figsize=(20, 20))
for i, col in enumerate(continuous_feature,1):
      df_copy = df.copy()
      plt.subplot(4, 2, i)
      sns.histplot(df_copy[col], bins=50, kde=True, color='darkblue')
      plt.title(f'Distribution of {col}')
      plt.xlabel(col)
      plt.ylabel('Count')

plt.tight_layout()
plt.show()

Most of the columns are skewed, so transformation is necessary.

In [ ]:
corr_matrix_con = df[continuous_feature].corr()
corr_matrix_con

These independent variables are highly correlated; we may want to perform feature selection.

In [ ]:
plt.figure(figsize=(10,20))

for i,col in enumerate(continuous_feature,1):
  df_copy = df.copy()
  plt.subplot (4,2,i)
  sns.boxplot(df_copy[col], color='darkblue')
  plt.title(f"Outlier of {col}")

plt.tight_layout()
plt.show()

Most columns have outliers. We need to be mindful of this when handling missing data, especially if there are any continuous features. Additionally, we need to apply some methods to handle these outliers.

**Temporal Variables (Datetime Variables)**

We identify datetime columns from object types, convert them to the correct datetime format, and then transform them into numeric columns.

In [ ]:
df.rename(columns= {'latest_transaction_booking_date': 'last_booking_date'}, inplace=True)

In [ ]:
from pandas.api.types import is_datetime64_any_dtype

date_col = []

for col in df.columns: 
    if df[col].dtypes == 'O':
      try:
         convert = pd.to_datetime(df[col])
         if is_datetime64_any_dtype(convert):
            if col != 'dob':
               new_name = col+'time'
               df.rename(columns={col:new_name}, inplace=True)
               date_col.append(new_name)
            else: 
              date_col.append(col)
      except Exception: 
         continue
print (date_col)

Since the data retrieves transactions within a 2-day period (April 2–4, 2025), we will not extract the month and year.

In [ ]:
new_date_cols = []
for col in date_col[:]:
    df[col] = pd.to_datetime(df[col])
    if col != 'dob':
      df[col + '_day'] = df[col].dt.day
      df[col + '_weekday'] = df[col].dt.day_of_week
      df[col + 'hour'] = df[col].dt.hour
      df[col + 'minute'] = df[col].dt.minute 
      df[col + 'sec'] = df[col].dt.second
      new_date_cols.extend([col + '_day', col + '_weekday'])

print(new_date_cols)

In [ ]:
plt.figure(figsize=(20,20))

for i,col in enumerate(new_date_cols,1):
    df_copy = df.copy
    plt.subplot(11,2,i)
    sns.countplot(x=col, data=df, palette='viridis')
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Count')

plt.tight_layout()
plt.show()

The columns `last_booking_datetime_day` and `last_booking_datetime_weekday` only have one unique value, so they provide no additional predictive power.

**Categorical Variables**

In [ ]:
categorical_features = df.select_dtypes(include='object').columns.to_list()

In [ ]:
#Identify the number of unique categories in each categorical (object) column.
for feature in categorical_features: 
    count_value = df[feature].nunique()
    print (feature,'-', count_value,'categories')

In [ ]:
df.to_csv('data/EDA_transaction_monitoring_merged.csv',index=False)